# Train notebook - Airbus kaggle challenge

# Ship Detection- Project AIRBUS 2019
## Model Parameters
We might want to adjust these later (or do some hyperparameter optimizations)

In [24]:
BATCH_SIZE = 64
EDGE_CROP = 16
GAUSSIAN_NOISE = 0.1
UPSAMPLE_MODE = 'SIMPLE'
# downsampling inside the network
NET_SCALING = None
# downsampling in preprocessing
IMG_SCALING_2 = (2,2)
IMG_SCALING_3 = (3,3)
# number of validation images to use
VALID_IMG_COUNT = 900
# maximum number of steps_per_epoch in training
MAX_TRAIN_STEPS = 10000
MAX_TRAIN_EPOCHS = 50

In [25]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from skimage.io import imread
from preprocess.pre_process import multi_rle_encode, rle_encode, rle_decode, masks_as_image, masks_as_color, balancing_train
from preprocess.pre_process import make_image_gen, create_aug_gen
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.utils import multi_gpu_model


import keras.backend as K
from keras.optimizers import Adam
from keras.losses import binary_crossentropy
from models.resnet50_classif import get_resnet50_classif

In [3]:
import tensorflow as tf
with tf.Session() as sess:
    devices = sess.list_devices()
for device in devices:
    print(device)
    
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")
    


_DeviceAttributes(/job:localhost/replica:0/task:0/device:CPU:0, CPU, 268435456, 18032800655511782370)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:0, XLA_GPU, 17179869184, 8215721929014167859)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:1, XLA_GPU, 17179869184, 7016826528577565619)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:2, XLA_GPU, 17179869184, 10178015169312792444)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_GPU:3, XLA_GPU, 17179869184, 11478384054601152313)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 4185320151801655778)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:0, GPU, 7392346112, 11797686308128496350)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:1, GPU, 7392346112, 1921370255297058797)
_DeviceAttributes(/job:localhost/replica:0/task:0/device:GPU:2, GPU, 7392346112, 11326282562495523068)
_DeviceAttributes(/job:localhost/

In [4]:
ship_dir = '../../data/airbus_ship_detection/'
train_image_dir = os.path.join(ship_dir, 'train')# Images for training
test_image_dir = os.path.join(ship_dir, 'test')# Images for testing
label_dir = os.path.join(ship_dir, 'train_ship_segmentations_v2.csv')# Images for testing
masks = pd.read_csv(label_dir, engine="python") # Markers for ships

In [5]:
label_has_ships = masks
label_has_ships['has_ship'] = masks.EncodedPixels.notnull()
label_has_ships = label_has_ships.drop(columns=['EncodedPixels'])
label_has_ships = label_has_ships.drop_duplicates()

In [6]:
count_has_ship = label_has_ships.has_ship.sum()
count_has_ship
print("rate:", count_has_ship/len(label_has_ships))

rate: 0.2210058372629261


In [7]:
len(label_has_ships)

192556

In [8]:
# data_balanced = label_has_ships.groupby('has_ship')
# data_balanced = data_balanced.apply(lambda x: x.sample(data_balanced.size().min()).reset_index(drop=True))
# data_balanced

In [9]:
# count_has_ship = data_balanced.has_ship.sum()
# count_has_ship
# print("rate:", count_has_ship/len(data_balanced))

In [10]:
# def make_image_gen(in_df, train_image_dir, batch_size=48, img_scalling=None):
#     all_batches = list(in_df.groupby('ImageId'))
#     out_rgb = []
#     out_label = []
#     while True:
#         np.random.shuffle(all_batches)
#         for c_img_id, c_label in all_batches:
#             rgb_path = os.path.join(train_image_dir, c_img_id)
#             c_img = imread(rgb_path)
            
#             if img_scalling is not None:
#                 c_img = c_img[::img_scalling[0], ::img_scalling[1]]
                
#             out_rgb.append(c_img)
#             out_label.append(c_label.has_ship.values[0])
#             if len(out_rgb)>=batch_size:
#                 yield np.stack(out_rgb, 0)/255.0, np.array(out_label)
#                 out_rgb, out_label=[], []   

In [11]:
training_set, validation_set = train_test_split(label_has_ships, test_size=0.05)
print(len(training_set))
print(len(validation_set))

182928
9628


In [14]:
train_gen = make_image_gen(training_set, train_image_dir, BATCH_SIZE, IMG_SCALING_3)
train_x, train_y = next(train_gen)
print('x', train_x.shape, train_x.min(), train_x.max())
print('y', train_y.shape, train_x.min(), train_x.max())

x (32, 256, 256, 3) 0.0 1.0
y (32,) 0.0 1.0


In [15]:
train_y

array([False, False, False, False,  True,  True, False, False,  True,
       False, False, False,  True, False, False, False, False, False,
        True, False,  True, False, False,  True, False, False, False,
       False, False, False, False, False])

### First step to train

In [28]:
model = get_resnet50_classif(input_shape=train_x.shape[1:])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 256, 256, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
flatten_3 (Flatten)          (None, 131072)            0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               16777344  
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 129       
Total params: 40,365,185
Trainable params: 40,312,065
Non-trainable params: 53,120
_________________________________________________________________


In [29]:
parallel_model = multi_gpu_model(model, gpus=4)

In [19]:
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard, Callback
weight_path = "../weights_models/{0}_weights_scaling{1}.hdf5".format('model_1',IMG_SCALING_3[0])

checkpoint = ModelCheckpoint(weight_path, monitor='val_loss', verbose=0, save_best_only=True, mode='min', save_weights_only=True)

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                   patience=2, verbose=1, mode='min',
                                   min_delta=0.001, cooldown=1, min_lr=1e-7)

early = EarlyStopping(monitor="val_loss", mode="min", verbose=2,
                      patience=2) # probably needs to be more patient, but kaggle time is limited

tensorboard =  TensorBoard(log_dir="../logs/log1", update_freq='batch')

callbacks_list = [reduceLROnPlat, early]



In [30]:
#run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)

parallel_model.compile(optimizer=Adam(), loss=binary_crossentropy, 
              metrics=["accuracy"])

step_count_train = min(MAX_TRAIN_STEPS, training_set.shape[0]//BATCH_SIZE)
training_gen = make_image_gen(training_set, train_image_dir, BATCH_SIZE, IMG_SCALING_3)
#training_aug_gen =  create_aug_gen(training_gen, image_gen, label_gen)

step_count_valid = validation_set.shape[0]//BATCH_SIZE
validation_gen = make_image_gen(validation_set, train_image_dir, BATCH_SIZE, IMG_SCALING_3)

print("step_count_train =", step_count_train)
print("step_count_valid =", step_count_valid)


loss_history = [parallel_model.fit_generator(training_gen,
                                 steps_per_epoch=step_count_train,
                                 epochs=MAX_TRAIN_EPOCHS,
                                 callbacks=callbacks_list,
                                 validation_data=validation_gen,
                                 validation_steps=step_count_valid)]

step_count_train = 2858
step_count_valid = 150
Epoch 1/50
 115/2858 [>.............................] - ETA: 1:46:19 - loss: 3.5169 - acc: 0.7789

KeyboardInterrupt: 

In [20]:
model.save("weights_models/{}_weights.best.hdf5".format('clf_95'))

In [ ]:
### Second step to train

In [ ]:
step_count_train = min(MAX_TRAIN_STEPS, training_set.shape[0]//BATCH_SIZE)
training_gen = make_image_gen(label_has_ships, train_image_dir, BATCH_SIZE, IMG_SCALING)
#training_aug_gen =  create_aug_gen(training_gen, image_gen, label_gen)

step_count_valid = validation_set.shape[0]//BATCH_SIZE
validation_gen = make_image_gen(validation_set, train_image_dir, BATCH_SIZE, IMG_SCALING)

print("step_count_train =", step_count_train)
print("step_count_valid =", step_count_valid)


loss_history = [model.fit_generator(training_gen,
                                 steps_per_epoch=step_count_train,
                                 epochs=5,
                                 callbacks=callbacks_list)]